<center><h2>Artificial and Computational Intelligence (Assignment - 2)</h2></center>

## Problem Statement

As part of the 2nd Assignment, we'll implement Bayesian Networks and also learn to use the pomegranate library.

You are required to create a bayesian network model which would help you predict the probability. The detailed problem description is attached as a PDF as a part of this assignment along with the marking scheme.  

### What is a Bayesian Network ?

A Bayesian network, Bayes network, belief network, decision network, Bayes(ian) model or probabilistic directed acyclic graphical model is a probabilistic graphical model (a type of statistical model) that represents a set of variables and their conditional dependencies via a directed acyclic graph (DAG). 

Bayesian networks are ideal for taking an event that occurred and predicting the likelihood that any one of several possible known causes was the contributing factor. For example, a Bayesian network could represent the probabilistic relationships between diseases and symptoms. Given symptoms, the network can be used to compute the probabilities of the presence of various diseases. 

### Dataset

The dataset can be downloaded from https://drive.google.com/drive/folders/1oMtKmmvPkN4O8DmrHMJe6M8CbB93Z5kw .You can access it only using your BITS IDs. Also, the same dataset is attached along with the assignment. 

#### Dataset Description
##### Sample Tuple

Y	won	5wickets	lost	2nd	vWest_Indies	Home	6-Nov-11

##### Explanation
- The first column represents if Ashwin was in the playing 11 or not. 
- The second column represents the Result of the match . win indicates India won the match.
- The third column represents the Margin of victory / losss.
- The fourth column represents the results of the toss. won indicates India won the toss. 
- The fifth column represents the batting order. If India batted 1st or 2nd. 
- The sixth column represents the opponent.
- The seventh column represents the location of the match. If the match was held in Home(India) or away. 
- The last column represents the start date of the match.


### Evaluation
We wish to evaluate based on 
- coding practices being followed
- commenting to explain the code and logic behind doing something
- your understanding and explanation of data
- how good the model would perform

In [ ]:
# BITS RollNumbers , Names. 

In [53]:
#Import libraries
import pandas as pd
import string
from pomegranate import *

In [4]:
#Read data
filename = 'India_Test_Stats.xlsx'
df = pd.read_excel(filename)

<bound method NDFrame.head of    Ashwin Result           Margin  Toss  Bat      Opposition Location  \
0       Y    won        5 wickets  lost  2nd   v West Indies     Home   
1       Y    won   inns & 15 runs   won  1st   v West Indies     Home   
2       Y   draw                -  lost  2nd   v West Indies     Home   
3       Y   lost         122 runs  lost  2nd     v Australia     Away   
4       Y   lost   inns & 68 runs   won  1st     v Australia     Away   
..    ...    ...              ...   ...  ...             ...      ...   
80      Y    won         203 runs   won  1st  v South Africa     Home   
81      Y    won  inns & 137 runs   won  1st  v South Africa     Home   
82      Y    won  inns & 202 runs   won  1st  v South Africa     Home   
83      Y    won  inns & 130 runs  lost  2nd    v Bangladesh     Home   
84      Y    won   inns & 46 runs  lost  2nd    v Bangladesh     Home   

   Start Date  
0  2011-11-06  
1  2011-11-14  
2  2011-11-22  
3  2011-12-26  
4  2012-01-03

In [50]:
# df.head
df.columns.to_list()

['Ashwin', 'Result', 'Toss', 'Bat', 'Location']

In [49]:
#Pre-process data (Whatever you feel might be required)
df = df.drop(columns = ['Margin', 'Opposition', 'Start Date'])

In [ ]:
#Data Description

In [ ]:
#Construction of Bayesian Network goes here 

In [58]:
#Solution for part 1 
def prior_prob(arr):
    mydict = dict()
    keys = list(set(arr))    
    nobs = len(arr)
    for i, val in enumerate(keys):
        mydict[keys[i]] = len(arr[arr == keys[i]]) / nobs
    return mydict


In [60]:
mydict = prior_prob(df['Ashwin'])
print(mydict)

{'N': 0.17647058823529413, 'Y': 0.8235294117647058}


In [33]:
#Solution for part 2 
def cond_prob(var_1, var_2):
    cond_prob_list = list()
    cat_1 = list(set(df[var_1]))     
    cat_2 = list(set(df[var_2]))    
    for i in cat_1:
        arr = df[df[var_1] == cat_1[0]][var_2]
        length = len(arr)
        for j in cat_2:
             cond_prob_list.append( [i, j, len(arr[arr == j]) / length] )
    return cond_prob_list     

In [34]:
var_1 = 'Ashwin'; var_2 = 'Result'
cond_prob_list = cond_prob(var_1, var_2)
print(cond_prob_list)

['draw', 'won', 'lost']
[['N', 'draw', 0.2], ['N', 'won', 0.3333333333333333], ['N', 'lost', 0.4666666666666667], ['Y', 'draw', 0.2], ['Y', 'won', 0.3333333333333333], ['Y', 'lost', 0.4666666666666667]]


In [61]:
#Solution for part 3 
"""
Columns in Data -
 'Ashwin',
 'Result',
 'Toss',
 'Bat',
 'Location',
"""
Ashwin = DiscreteDistribution(prior_prob(df['Ashwin']))
Toss = DiscreteDistribution(prior_prob(df['Toss']))
Bat = ConditionalProbabilityTable(cond_prob(df['Toss'], df['Bat']))
Location = DiscreteDistribution(prior_prob(df['Location']))
Result = ConditionalProbabilityTable(cond_prob(df['Ashwin'], df['Bat'], df['Location'], df['Result']))

node_Ashwin = Node(Ashwin, name="Ashwin")
node_Toss = Node(Toss, name="Toss")
node_Bat = Node(Bat, name="Bat")
node_Location = Node(Location, name="Location")
node_Result = Node(Result, name="Result")

model = BayesianNetwork("Selectors Conundrum")
model.add_states(node_Ashwin, node_Location, node_Toss, node_Bat, node_Result)
model.add_edge(node_Ashwin, node_Result)
model.add_edge(node_Toss, node_Bat)
model.add_edge(node_Bat, node_Result)
model.add_edge(node_Location, node_Result)
model.bake()

In [ ]:
#Solution for part 4
#['Ashwin', 'Toss', Bat', 'Location', Result']
# a)
solution_a = sum( model.probability([['Y', 'lost' '2nd', 'Home', 'won'],
                                     ['Y', 'won' '2nd', 'Home', 'won'],
                                     ['Y', 'lost' '2nd', 'Away', 'won'],
                                     ['Y', 'won' '2nd', 'Away', 'won']]) )
print(solution_a)

# b)
solution_b = sum( model.probability([['N', 'lost' '2nd', 'Home', 'won'],
                                     ['N', 'won' '2nd', 'Home', 'won'],
                                     ['N', 'lost' '2nd', 'Away', 'won'],
                                     ['N', 'won' '2nd', 'Away', 'won']]) )
print(solution_b)

# c)
solution_c = sum( model.probability([['Y', 'lost' '2nd', 'Home', 'lost'],
                                     ['Y', 'won' '2nd', 'Home', 'lost'],
                                     ['Y', 'lost' '2nd', 'Away', 'lost'],
                                     ['Y', 'won' '2nd', 'Away', 'lost']]) )
print(solution_c)

# d) 
solution_d = sum( model.probability([['N', 'lost' '2nd', 'Home', 'lost'],
                                     ['N', 'won' '2nd', 'Home', 'lost'],
                                     ['N', 'lost' '2nd', 'Away', 'lost'],
                                     ['N', 'won' '2nd', 'Away', 'lost']]) )
print(solution_d)

In [ ]:
#Feel free to add cells where necessary. 

<h3><center> Happy Coding!</center></h3>